In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
from scipy.stats import norm
#%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

class Clock_Animation: #класс для рисования
    def __init__(self, U0, psi):
        self.U0 = U0
        self.psi = psi
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(350, 550), ylim=(-.002, .002))
        self.line, = self.ax.plot([], [], lw=2)
        #self.line_potential, = self.ax.plot([], [], lw=2) #потенциал
        
        #Для корректной работы установить ffmpeg 
        #Для среды notebook вбить в anaconda: conda install -c conda-forge ffmpeg
        anim = animation.FuncAnimation(self.fig, self.animate, init_func=self.initialization_func, 
                                       frames=2000, interval=20, blit=True)
        anim.save('Clock.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
        plt.show()
    
    #initialization function: plot the background of each frame
    def initialization_func(self):
        self.line.set_data([], [])
        #self.line_potential.set_data([], [])
        return self.line,
    
    # animation function.  This is called sequentially
    def animate(self, i):
        A = self.psi[i % (self.psi.shape[0])] # '%' для зацикливания анимации
        
        x = np.arange(0, self.psi.shape[0])
        y = A*np.conjugate(A)
        self.line.set_data(x, y)
        return self.line,

    
class Clock:
    def __init__(self, Lt, U0, psi_profile, Tmax = 1.):
        self.hx = 1./(U0.size - 1) #шаг по координате
        self.Lt = Lt #шаг по времени
                
        self.U0 = U0
        
        self.Nx = int(U0.size)
        self.Nt = int(Tmax/Lt + 1)
        self.psi = np.zeros(shape=(self.Nt, self.Nx), dtype = complex)
        
        self.Lt = .0001 #шаг по времени
        
        #Считаем, что на границах в.ф. точный ноль всегда
        self.psi[:, 0] = 0. 
        self.psi[:,-1] = 0.
        
        self.psi[0] = psi_profile #начальное распределение
        
        #Вместо первого шага по явной схеме скопируем исходный профиль
        self.psi[1] = psi_profile
        
        self.Dufert_Frankel_scheme()
               
        clock_anim = Clock_Animation(U0, self.psi)
        
    def Dufert_Frankel_scheme(self):
        for i in range(1, self.Nt - 1):
            self.psi[i+1, 1:self.Nx-1] = (self.psi[i, 2:self.Nx]/self.hx**2 + self.psi[i, 0:self.Nx-2]/self.hx**2 -\
                                        self.psi[i-1, 1:self.Nx-1]*(1./hx**2 + U0[1:self.Nx-1]/2. + 1j/(2.*self.Lt)))*\
                                        (1/self.hx**2 + U0[1:self.Nx-1]/2. - 1j/(2.*self.Lt))**(-1)
        
        np.set_printoptions(threshold=np.nan)
        #print self.psi[:,300]*np.conjugate(self.psi[:,300])

         #Psi[1:Nx-1,i+1] = (Psi[1:Nx-1,i-1]*(Um[1:Nx-1]*dt/1j + eta/1j + 1)\
      # - eta/1j*(Psi[2:,i] + Psi[:Nx-2,i]))/(1 - eta/1j - Um[1:Nx-1]*dt/1j)
    #Psi[:,i+1] = Psi[:,i+1]/np.sqrt(norma(Psi[:,i+1], xarr))


        



Lt = .001 
hx = .001 
size = 1001
ll = 400
lr = 450
rl = 452
rr = 502

#U0 = np.zeros(size)
U0 = np.ones(size)*100000
U0[ll:lr] = -10.
U0[rl:rr] = -10.


sigma = 20.
mu = (ll + lr)/2
psi_profile = np.zeros(size)
psi_profile[ll:lr] = norm(mu, sigma).pdf(np.arange(ll, lr)) #гауссов пакет в левой яме

#print psi_profile[ll:lr].sum()
#print psi_profile[ll:lr]
Clock(Lt, U0, psi_profile)